In [ ]:
-- Creates Calendar Objects for Calcuation of Calendar.
CREATE PROCEDURE DM.usp_CalendarObjectsCreate AS
-- EXEC Flavors_DW_Staging.DM.usp_CalendarObjectsCreate
BEGIN

-- ########################
-- Create Calendar Tables for Loading
-- ########################

IF OBJECT_ID(N'dbo.dimHoliday', N'U') IS NOT NULL  DROP TABLE [dbo].[dimHoliday]

CREATE TABLE [dbo].[dimHoliday](
	[RowId] [INT] IDENTITY(1,1) NOT NULL,
	[DateKey] [INT] NULL,
	[HolidayYear] [INT] NULL,
	[HolidayDate] [DATETIME] NULL,
	[Location] [VARCHAR](30) NULL,
	[HolidayDescription] [VARCHAR](30) NULL,
 CONSTRAINT [PK_dimHoliday] PRIMARY KEY CLUSTERED 
(
	[RowId] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY]


-- Load Calendar Table From DIM DATE
INSERT INTO [Flavors_DW_Staging].[dbo].[dimHoliday] ([DateKey], [HolidayYear], [HolidayDate], [Location], [HolidayDescription])
SELECT CONVERT(VARCHAR(10),[DATE],112) AS [DateKey], [Year], CONVERT(DATE,[DATE]) AS HolidayDate, 'USA' AS [Location], HolidayUSA 
FROM Flavors_DW.DM.DIM_DATE 
WHERE IsHolidayUSA = 1;

INSERT INTO [Flavors_DW_Staging].[dbo].[dimHoliday] ([DateKey], [HolidayYear], [HolidayDate], [Location], [HolidayDescription])
SELECT CONVERT(VARCHAR(10),[DATE],112) AS [DateKey], [Year], CONVERT(DATE,[DATE]) AS HolidayDate, 'UK' AS [Location], HolidayUSA 
FROM Flavors_DW.DM.DIM_DATE 
WHERE IsHolidayUK = 1;


------ Create Date Table -------------------------
IF OBJECT_ID(N'dbo.dimCalendar', N'U') IS NOT NULL  DROP TABLE [dbo].[dimCalendar]

CREATE TABLE [dbo].[dimCalendar](
	[DateKey] [INT] NOT NULL,
	[Date] [DATE] NOT NULL,
	[Day] [TINYINT] NULL,
	[Weekday] [TINYINT] NULL,
	[WeekDayName] [VARCHAR](10) NULL,
	[IsWeekend] [TINYINT] NULL,
	[IsHoliday] [TINYINT] NULL,
	[HolidayText] [VARCHAR](60) NULL,
	[DayOfYear] [SMALLINT] NULL,
	[WeekOfMonth] [TINYINT] NULL,
	[WeekOfYear] [TINYINT] NULL,
	[ISOWeekOfYear] [TINYINT] NULL,
	[Month] [TINYINT] NULL,
	[MonthName] [VARCHAR](10) NULL,
	[Quarter] [TINYINT] NULL,
	[QuarterName] [VARCHAR](6) NULL,
	[Year] [INT] NULL,
	[WeekNo] [VARCHAR](7) NULL,
	[MonthNo] [VARCHAR](7) NULL,
	[QuaterlyNo] [VARCHAR](7) NULL,
	[IsWorkDay] [INT] NOT NULL,
	[FirstDayOfWeek] [DATE] NULL,
	[LastDayOfWeek] [DATE] NULL,
	[FirstDayOfMonth] [DATE] NULL,
	[LastDayOfMonth] [DATE] NULL,
	[MonthWorkDay] [INT] NULL,
	[MonthWorkLeft] [INT] NULL,
	[MonthWorkTotal] [INT] NULL,
	[FirstDayOfQuarter] [DATE] NULL,
	[LastDayOfQuarter] [DATE] NULL,
	[QuarterWorkDay] [INT] NULL,
	[QuarterWorkLeft] [INT] NULL,
	[QuarterWorkTotal] [INT] NULL,
	[FirstDayOfYear] [DATE] NULL,
	[LastDayOfYear] [DATE] NULL,
	[YearWorkDay] [INT] NULL,
	[YearWorkLeft] [INT] NULL,
	[YearWorkTotal] [INT] NULL,
	[CalendarDate] [DATETIME] NULL,
	[IsHolidayUK] [TINYINT] NULL,
	[HolidayTextUK] [VARCHAR](60) NULL,
	[IsWorkDayUK] [INT] NOT NULL,
	[WeekdayUK] [TINYINT] NULL,
	DayOfWeekInMonth  [TINYINT] NULL,
	Workday_Date_DWKey [INT] NULL,
	DayOfWeekInYear [TINYINT] NULL,
    DayOfQuarter [TINYINT] NULL,
    WeekOfMonth2 [TINYINT] NULL,
    WeekOfQuarter [TINYINT] NULL,
    WeekOfYear2 [TINYINT] NULL,
    MonthOfQuarter [TINYINT] NULL,
    FiscalWeekOfYear [TINYINT] NULL,
    Day1MonthEnd [TINYINT] NULL,
    InvoiceDay [TINYINT] NULL,
    Work_Day [TINYINT] NULL,
	WeekEndingDate_Sunday [DATE] NULL, 
	FiscalFirstDayOfMonth [DATE] NULL,
	FiscalLastDayOfMonth [DATE] NULL,
	FiscalFirstDayOfQuarter [DATE] NULL,
	FiscalLastDayOfQuarter [DATE] NULL,
	FiscalFirstDayOfYear [DATE] NULL,
	FiscalLastDayOfYear [DATE] NULL,
 CONSTRAINT [PK_dimCalendar] PRIMARY KEY CLUSTERED 
(
	[Date] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY];


-- ######################## 
-- Remove if exists Functions
-- ########################
IF OBJECT_ID(N'dbo.ufnGetWorkDaysbyRange', N'FN') IS NOT NULL  DROP FUNCTION [dbo].[ufnGetWorkDaysbyRange];
IF OBJECT_ID(N'dbo.ufnIsWorkDay', N'FN') IS NOT NULL  DROP FUNCTION dbo.ufnIsWorkDay;
IF OBJECT_ID(N'dbo.ufnIsHoliday', N'FN') IS NOT NULL  DROP FUNCTION dbo.ufnIsHoliday;
IF OBJECT_ID(N'dbo.ufnNextWorkdays', N'FN') IS NOT NULL  DROP FUNCTION dbo.ufnNextWorkdays;
IF OBJECT_ID(N'dbo.ufnGetHolidayText', N'FN') IS NOT NULL  DROP FUNCTION dbo.ufnGetHolidayText;
IF OBJECT_ID(N'dbo.ufnPreviousWorkdays', N'FN') IS NOT NULL  DROP FUNCTION dbo.ufnPreviousWorkdays;
DECLARE @TSQL VARCHAR(MAX)

-- ######################## 
-- Create Functions Objects for Calendar Stuff...
-- ########################

SET @TSQL = '
-- ##### Function number of work days by range for a location.
CREATE FUNCTION [dbo].[ufnGetWorkDaysbyRange] (@DateStart DATETIME,@DateEnd DATETIME,@Location VARCHAR(30))
RETURNS INT
AS
  BEGIN
  DECLARE @OrgStartDate DATETIME = @DateStart

      IF ( @DateStart IS NULL OR @DateEnd IS NULL )
      RETURN ( 0 )
      
      DECLARE @i INT = 0;
      DECLARE @h INT = 0;
      
--Looping days and counting them, do not count Sat & Sundays
      WHILE ( @DateStart <= @DateEnd )
        BEGIN
            SET @i = @i + CASE
                          WHEN DATENAME(dw, @DateStart) IN (''Saturday'', ''Sunday'' ) THEN 0
                          ELSE 1
                          END
            -- Inc the Start date until not true.
            SET @DateStart = @DateStart + 1
        END
-- end while loop  
        
        -- Count Holidays
        SET @h = ISNULL((SELECT COUNT(*) AS CNT FROM dimHoliday WHERE [Location] = @Location AND DATENAME(dw,HolidayDate) NOT IN (''Saturday'', ''Sunday'' )  AND (HolidayDate >= @OrgStartDate AND @DateEnd >= HolidayDate)),0) 
		
        
        -- Subtrack Holidays f
        SET @i = @i - @h
        
      RETURN ( @i )
  END ;
  '
 EXEC(@TSql);

SET @TSQL = '
-- ##### Function Confrim date is a work by location is a work day or not.
CREATE FUNCTION [dbo].[ufnIsWorkDay] (@InputDate DATETIME, @Location VARCHAR(30))
RETURNS BIT
AS
BEGIN  
DECLARE @MyReturn BIT = 0

IF (DATEPART(dw, @InputDate) BETWEEN 2 AND 6) AND 
      (@InputDate NOT IN 
          (
           SELECT HolidayDate 
           FROM dimHoliday
           WHERE [HolidayYear] = DATEPART(yyyy,@InputDate) AND [Location] = @Location 
         )
       )
    BEGIN
		SET @MyReturn = 1
    END
    RETURN @MyReturn 
END ;
  '
 EXEC(@TSql);


 SET @TSQL = '
-- ##### Function is location date is a holiday or not.
CREATE FUNCTION [dbo].[ufnIsHoliday] (@InputDate DATETIME, @Location VARCHAR(30))
RETURNS BIT
AS
BEGIN  
DECLARE @MyReturn BIT = 0
DECLARE @CNT BIT = 0

SELECT @CNT = COUNT(*)  FROM dimHoliday WHERE [HolidayDate] = @InputDate AND [Location] = @Location 
 IF @CNT >= 1 
    BEGIN
		SET @MyReturn = 1
    END
    
    RETURN @MyReturn
       
END ;
  '
 EXEC(@TSql);

 SET @TSQL = '
-- ##### Function Gives you the next work day for a given date by location.
CREATE FUNCTION [dbo].[ufnNextWorkdays] (@startdate DATETIME, @days INT, @Location VARCHAR(30) )
   RETURNS DATETIME
AS
BEGIN 
DECLARE @enddate DATETIME
DECLARE @counter INT
SET @counter = 1
SET @enddate = @startdate

WHILE @counter <= @days
BEGIN
SET @enddate = DATEADD(dd, 1, @enddate)

   IF (DATEPART(dw, @enddate) BETWEEN 2 AND 6) AND 
      (@enddate NOT IN 
          (
           SELECT HolidayDate 
           FROM dimHoliday
           WHERE [HolidayYear] = DATEPART(yyyy,@enddate) AND [Location] = @Location 
         )
       )
   BEGIN
      SET @counter = @counter + 1
   END
END
RETURN @enddate
END;
  '
 EXEC(@TSql);


SET @TSQL = '
-- ##### Function Get Holiday Text
CREATE FUNCTION [dbo].[ufnGetHolidayText] (@InputDate DATETIME, @Location VARCHAR(30))
RETURNS VARCHAR(30)
AS
BEGIN  
DECLARE @MyReturn VARCHAR(30) = ''''
DECLARE @CNT BIT = 0

SELECT @MyReturn = HolidayDescription FROM dimHoliday WHERE [HolidayDate] = @InputDate AND [Location] = @Location 

    RETURN @MyReturn   
END ;
  '
 EXEC(@TSql);

SET @TSQL = '
-- ##### Function Gives you the previous work day for a given date by location.
CREATE FUNCTION [dbo].[ufnPreviousWorkdays] (@startdate DATETIME, @days INT, @Location VARCHAR(30))
   RETURNS DATETIME
AS
BEGIN 
DECLARE @enddate DATETIME
DECLARE @counter INT
SET @counter = 1
SET @enddate = @startdate

WHILE @counter <= @days

BEGIN

SET @enddate = DATEADD(dd, -1, @enddate)

   IF (DATEPART(dw, @enddate) BETWEEN 2 AND 6) AND 
      (@enddate NOT IN 
          (
           SELECT HolidayDate 
           FROM dimHoliday
           WHERE [HolidayYear] = DATEPART(yyyy,@enddate) AND [Location] = @Location 
         )
       )
   BEGIN
      SET @counter = @counter + 1
   END
END
RETURN @enddate
END;
  '
 EXEC(@TSql);


END;

In [ ]:
-- Generate and Populate Calendar
CREATE PROCEDURE dbo.usp_CalendarYearAppend  (@VarYear INT, @InsertYN CHAR(1)) AS
-- EXEC DM.usp_CalendarYearAppend 2023, 'Y'
DECLARE @CNT INT;

-- DEBUGGING VARIABLES...
DECLARE @VarYear INT = 2023
--DECLARE @InsertYN Char = 'N'

-- Check Objects
IF OBJECT_ID(N'dbo.dimHoliday', N'U') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.dimHoliday please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.dimCalendar', N'U') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.dimCalendar please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnGetWorkDaysbyRange', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnGetWorkDaysbyRange please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnIsWorkDay', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnIsWorkDay please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnIsHoliday', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnIsHoliday please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnNextWorkdays', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnNextWorkdays please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnGetHolidayText', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnGetHolidayText please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END
IF OBJECT_ID(N'dbo.ufnPreviousWorkdays', N'FN') IS NULL BEGIN PRINT 'MISSING OBJECT dbo.ufnPreviousWorkdays please run to fix EXEC DM.usp_CalendarObjectsCreate' RETURN END

-- Check if we have enough vacation time off 
SELECT @CNT = COUNT(*) FROM dbo.dimHoliday WHERE HolidayYear = @VarYear

--Do we have vacation time in that year?
IF 112 > @CNT 
BEGIN
PRINT '------------------------------------';
PRINT ' Only found '+ CONVERT(VARCHAR(2),ISNULL(@CNT,0)) + 'days of vacation which does not seem like enough vacation time off. ';
PRINT '------------------------------------';
PRINT '';
PRINT '----------------[ EXAMPLE TO INSERT ]--------------';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230102'',''2023'',''1/2/2023'',''USA'',''New Years Day (observed)''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230116'',''2023'',''1/16/2023'',''USA'',''Martin Luther King Jr. Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230407'',''2023'',''4/7/2023'',''USA'',''Good Friday''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230529'',''2023'',''5/29/2023'',''USA'',''Memorial Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230619'',''2023'',''6/19/2023'',''USA'',''Juneteenth Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230703'',''2023'',''7/3/2023'',''USA'',''Day Before Independence Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230704'',''2023'',''7/4/2023'',''USA'',''Independence Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20230904'',''2023'',''9/4/2023'',''USA'',''Labor Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20231110'',''2023'',''11/10/2023'',''USA'',''Veterans Day (observed)''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20231123'',''2023'',''11/23/2023'',''USA'',''Thanksgiving Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20231124'',''2023'',''11/24/2023'',''USA'',''Day After Thanksgiving''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20231225'',''2023'',''12/25/2023'',''USA'',''Christmas Day''';
PRINT 'Insert into [dbo].[dimHoliday] (DateKey,HolidayYear,HolidayDate,Location,HolidayDescription)  Select ''20231226'',''2023'',''12/26/2023'',''USA'',''Day after Christmas''';
RETURN
END



-- ########################################################################
-- ######## Begin Calendar Generation ###############
IF OBJECT_ID('tempdb..#dimCal', 'U') IS NOT NULL DROP TABLE #dimCal;

DECLARE 
@StartDate DATE = CONCAT(@VarYear,'0101'), 
@NumberOfYears INT = 1;
SET DATEFIRST 7;
SET DATEFORMAT mdy;
SET LANGUAGE US_ENGLISH;

DECLARE @CutoffDate DATE = DATEADD(YEAR, @NumberOfYears, @StartDate);

-- Temp Table...
CREATE TABLE #dimCal
(
  [date]       DATE PRIMARY KEY, 
  [day]        AS DATEPART(DAY,      [date]),
  [month]      AS DATEPART(MONTH,    [date]),
  FirstOfMonth AS CONVERT(DATE, DATEADD(MONTH, DATEDIFF(MONTH, 0, [date]), 0)),
  FirstOfWeek  AS CONVERT(DATE, DATEADD(Week, DATEDIFF(WEEK, 0, [date]), 0)),
  [MonthName]  AS DATENAME(MONTH,    [date]),
  [week]       AS DATEPART(WEEK,     [date]),
  [ISOweek]    AS DATEPART(ISO_WEEK, [date]),
  [DayOfWeek]  AS DATEPART(WEEKDAY,  [date]),
  [quarter]    AS DATEPART(QUARTER,  [date]),
  [year]       AS DATEPART(YEAR,     [date]),
  FirstOfYear  AS CONVERT(DATE, DATEADD(YEAR,  DATEDIFF(YEAR,  0, [date]), 0)),
  Style112     AS CONVERT(CHAR(8),   [date], 112),
  Style101     AS CONVERT(CHAR(10),  [date], 101)
);

INSERT #dimCal([date]) 

SELECT d
FROM
(
  SELECT d = DATEADD(DAY, rn - 1, @StartDate)
  FROM 
  (
    SELECT TOP (DATEDIFF(DAY, @StartDate, @CutoffDate)) 
      rn = ROW_NUMBER() OVER (ORDER BY s1.[object_id])
    FROM sys.all_objects AS s1
    CROSS JOIN sys.all_objects AS s2
    ORDER BY s1.[object_id]
  ) AS x
) AS y;


------ Full Table clear and reload --------------
DELETE FROM [dbo].[dimCalendar] 

------ INSERT --------------
INSERT INTO [dbo].[dimCalendar]
                         ([DateKey], [Date], [Day], [Weekday], [WeekdayUK], [WeekDayName], [IsWeekend], [IsHoliday], [HolidayText],  [IsHolidayUK], HolidayTextUK, [DayOfYear], [WeekOfMonth], [WeekOfYear], [ISOWeekOfYear], [Month], [MonthName], [Quarter], [QuarterName], [Year], [WeekNo], [MonthNo], 
                         [QuaterlyNo], [IsWorkDay], IsWorkDayUK,[FirstDayOfWeek], [LastDayOfWeek],[FirstDayOfMonth], [LastDayOfMonth], [MonthWorkDay], [MonthWorkLeft], [MonthWorkTotal], [FirstDayOfQuarter], [LastDayOfQuarter], [QuarterWorkDay], [QuarterWorkLeft], [QuarterWorkTotal], 
                         [FirstDayOfYear], [LastDayOfYear], [YearWorkDay], [YearWorkLeft], [YearWorkTotal],CalendarDate,DayOfWeekInYear,WeekEndingDate_Sunday, DayOfQuarter) 
SELECT 
  [DateKey]       = CONVERT(CHAR(8),[date],112),
  [Date]          = CONVERT(DATE,[DATE]),
  [Day]           = CONVERT(TINYINT, [day]),
  [Weekday]       = CONVERT(TINYINT, [DayOfWeek]),
  [WeekdayUK]     = CONVERT(TINYINT,  ((datepart(DW, [DATE]) + @@DATEFIRST + 6) % 7) + 1 ), -- Will need to set each year
  [WeekDayName]   = CONVERT(VARCHAR(10), DATENAME(WEEKDAY, [date])),
  [IsWeekend]     = CONVERT(TINYINT, CASE WHEN [DayOfWeek] IN (1,7) THEN 1 ELSE 0 END),
  [IsHoliday]     = dbo.ufnIsHoliday([date],'USA'),
  HolidayText     = dbo.ufnGetHolidayText([date],'USA'),
  [IsHolidayUK]   = dbo.ufnIsHoliday([date],'UK'),
  HolidayTextUK   = dbo.ufnGetHolidayText([date],'UK'),
  [DayOfYear]     = CONVERT(SMALLINT, DATEPART(DAYOFYEAR, [date])),
  WeekOfMonth     = CONVERT(TINYINT, DENSE_RANK() OVER (PARTITION BY [year], [month] ORDER BY [week])),
  WeekOfYear      = CONVERT(TINYINT, [week]),
  ISOWeekOfYear   = CONVERT(TINYINT, ISOWeek),
  [Month]         = CONVERT(TINYINT, [month]),
  [MonthName]     = CONVERT(VARCHAR(10), [MonthName]),
  [Quarter]       = CONVERT(TINYINT, [quarter]),
  QuarterName     = CONVERT(VARCHAR(6), CASE [quarter] WHEN 1 THEN 'First'  WHEN 2 THEN 'Second' WHEN 3 THEN 'Third' WHEN 4 THEN 'Fourth' END), 
  [Year]          = [year],
  [WeekNo]        = CAST([year] AS CHAR(4))+'-'+RIGHT('00'+CAST(CONVERT(TINYINT, ISOWeek) AS VARCHAR(2)), 2),
  [MonthNo]       = CAST([year] AS CHAR(4))+'-'+RIGHT('00'+CAST(CONVERT(TINYINT, [month]) AS VARCHAR(2)), 2),
  [QuaterlyNo]    = CAST([year] AS CHAR(4))+'-'+RIGHT('00'+CAST(CONVERT(TINYINT, [quarter]) AS VARCHAR(2)), 2),
  IsWorkDay       = dbo.ufnIsWorkDay([date],'USA'),
  IsWorkDayUK     = dbo.ufnIsWorkDay([date],'UK'),

  ------------ WEEK ---------------------
  FirstDayOfWeek  = FirstOfWeek,
  LastDayOfWeek   = DateAdd(d,6,FirstOfWeek),
  ------------ MONTH ---------------------
  FirstDayOfMonth     = FirstOfMonth,
  LastDayOfMonth      = MAX([date]) OVER (PARTITION BY [year], [month]),
  MonthWorkDay    = CASE 
 	            	WHEN (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')-1) - (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,[date],'USA') - 1) = 0
 	             	THEN (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')-1)
 		            ELSE ((dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')-1) - (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')-1) -  (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,[date],'USA') - 0) * -1) END,
  MonthWorkLeft =  (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')-0) - (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,[date],'USA') - 1),
  MonthWorkTotal = (dbo.ufnGetWorkDaysbyRange(FirstOfMonth,MAX([date]) OVER (PARTITION BY [year], [month]),'USA')), 
  ------------ Quarter ---------------------
  FirstDayOfQuarter   = MIN([date]) OVER (PARTITION BY [year], [quarter]),
  LastDayOfQuarter    = MAX([date]) OVER (PARTITION BY [year], [quarter]),
  QuarterWorkDay  = CASE 
 	            	WHEN (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')-1) - (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),[date],'USA') - 1) = 0
 	             	THEN (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')-1)
 		            ELSE ((dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')-1) - (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')-1) -  (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),[date],'USA') - 0) * -1) END,
  QuarterWorkLeft =  (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')-0) - (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),[date],'USA') - 1),
  QuarterWorkTotal = (dbo.ufnGetWorkDaysbyRange(MIN([date]) OVER (PARTITION BY [year], [quarter]),MAX([date]) OVER (PARTITION BY [year], [quarter]),'USA')), 
  ------------ Year ---------------------
  FirstDayOfYear      = FirstOfYear,
  LastDayOfYear       = MAX([date]) OVER (PARTITION BY [year]),
  YearWorkDay     = CASE 
		        	WHEN (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-1) -  (dbo.ufnGetWorkDaysbyRange(FirstOfYear,date,'USA')-1) = 0
		        	THEN (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-1)
		        	ELSE (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-1) - (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-1) -  (dbo.ufnGetWorkDaysbyRange(FirstOfYear,date,'USA')-1)  *-1 END,
  YearWorkLeft =  (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-1) -  (dbo.ufnGetWorkDaysbyRange(FirstOfYear,date,'USA')-1) ,
  YearWorkTotal = (dbo.ufnGetWorkDaysbyRange(FirstOfYear, MAX([date]) OVER (PARTITION BY [year]),'USA')-0)

  ------------ Calendar Day only for checking ---------------------
   ,CASE
	WHEN dbo.ufnIsWorkDay([date],'USA') = 1 THEN [Date]
	WHEN dbo.ufnIsWorkDay([date],'USA') = 0 and  Convert(varchar(5),[date],101) = '01/01' THEN dbo.ufnNextWorkdays([date],1,'USA')
	ELSE 
	dbo.ufnPreviousWorkdays([date],1,'USA')
	END CalendarDate,

	CONVERT(TINYINT,  ((datepart(DW, [Date]) + @@DATEFIRST + 7) % 7) + 1 ) AS DayOfWeekInYear

   ,CASE
	WHEN CONVERT(VARCHAR(10), DATENAME(WEEKDAY, [date])) = 'Sunday' THEN [date]
	ELSE DATEADD(DAY , 8-DATEPART(WEEKDAY,[Date]),[Date]) 
	END AS WeekEndingDate_Sunday,
	DATEDIFF(DAY, DATEADD(QUARTER, DATEDIFF(QUARTER, 0, [Date]), 0), [Date]) + 1 AS DayOfQuarter--,

FROM #dimCal
Order by [Date];


--Corrections
Update [dimCalendar]
Set QuarterWorkDay = 0, QuarterWorkLeft = QuarterWorkTotal 
Where 0 > QuarterWorkDay; 

Update  [dimCalendar]
Set MonthWorkDay = 0, MonthWorkLeft = MonthWorkTotal
Where 0 > MonthWorkDay;

Update  [dimCalendar]
Set YearWorkDay = 0, YearWorkLeft = YearWorkTotal
Where 0 > YearWorkDay;

Update [dimCalendar]
Set MonthWorkLeft = MonthWorkTotal
Where MonthWorkLeft > MonthWorkTotal;


-- ########################################################################
-- Insert into Calendar Table
-- You can adjust the table and select to fit your requirements.
-- ########################################################################


IF @InsertYN = 'Y'
BEGIN 
IF OBJECT_ID(N'dbo.DIM_DATE', N'U') IS NOT NULL BEGIN DROP TABLE DBO.DIM_DATE END
--DELETE FROM DBO.DIM_DATE  WHERE [Year] = @VarYear;

--INSERT INTO  DBO.DIM_DATE (DateKey, Date, FullDateUK, FullDateUSA, DayOfMonth, DaySuffix, DayName, DayOfWeekUSA, DayOfWeekUK, DayOfWeekInMonth, DayOfWeekInYear, DayOfQuarter, DayOfYear, WeekOfMonth, WeekOfQuarter, WeekOfYear, 
--                         WeekEndingDate_Sunday, Month, MonthName, MonthOfQuarter, Quarter, QuarterName, Year, YearName, MonthYear, MMYYYY, FirstDayOfMonth, LastDayOfMonth, FirstDayOfQuarter, LastDayOfQuarter, FirstDayOfYear, 
--                         LastDayOfYear, IsHolidayUSA, IsWeekday, HolidayUSA, IsHolidayUK, HolidayUK, FiscalDayOfYear, FiscalWeekOfYear, FiscalMonth, FiscalQuarter, FiscalQuarterName, FiscalYear, FiscalYearName, FiscalMonthYear, 
--                         FiscalMMYYYY, FiscalFirstDayOfMonth, FiscalLastDayOfMonth, FiscalFirstDayOfQuarter, FiscalLastDayOfQuarter, FiscalFirstDayOfYear, FiscalLastDayOfYear, PERIOD_CODE, Day1MonthEnd, InvoiceDay, 
--                         WORKDAY_DATE_DWKEY, WORK_DAY)
SELECT DateKey, Date, FullDateUK, FullDateUSA, DayOfMonth, DaySuffix, DayName, DayOfWeekUSA, DayOfWeekUK, DayOfWeekInMonth, DayOfWeekInYear, DayOfQuarter, DayOfYear, WeekOfMonth, WeekOfQuarter, WeekOfYear, 
                         WeekEndingDate_Sunday, Month, MonthName, MonthOfQuarter, Quarter, QuarterName, Year, YearName, MonthYear, MMYYYY, FirstDayOfMonth, LastDayOfMonth, FirstDayOfQuarter, LastDayOfQuarter, FirstDayOfYear, 
                         LastDayOfYear, IsHolidayUSA, IsWeekday, HolidayUSA, IsHolidayUK, HolidayUK, FiscalDayOfYear, FiscalWeekOfYear, FiscalMonth, FiscalQuarter, FiscalQuarterName, FiscalYear, FiscalYearName, FiscalMonthYear, 
                         FiscalMMYYYY, FiscalFirstDayOfMonth, FiscalLastDayOfMonth, FiscalFirstDayOfQuarter, FiscalLastDayOfQuarter, FiscalFirstDayOfYear, FiscalLastDayOfYear, PERIOD_CODE, Day1MonthEnd, InvoiceDay, 
                         WORKDAY_DATE_DWKEY, WORK_DAY
						 --,[Space], ISOWeekOfYear,  WeekNo, MonthNo, QuaterlyNo,  IsWeekend, FirstDayOfWeek, LastDayOfWeek,  MonthWorkDay, MonthWorkLeft, MonthWorkTotal, QuarterWorkDay, QuarterWorkLeft, QuarterWorkTotal, YearWorkDay, YearWorkLeft, YearWorkTotal, CalendarDate
INTO DBO.DIM_DATE -- UnComment if you already have Date Calendar Table
FROM (
SELECT   
DateKey, CONVERT(DATETIME,DATE,101) AS [Date], 
CONVERT(VARCHAR(10), DATE,101) AS FullDateUK, 
CONVERT(VARCHAR(10), DATE,101)AS FullDateUSA,
Day AS DayOfMonth, 

CASE 
WHEN LEN(Day) = 2 AND LEFT(Day,1) = 1 THEN Convert(varchar(2),Day)+'th'
WHEN RIGHT(Day,1) = 1 THEN Convert(varchar(2),Day)+'st'
WHEN RIGHT(Day,1) = 2 THEN Convert(varchar(2),Day)+'nd'
WHEN RIGHT(Day,1) = 3 THEN Convert(varchar(2),Day)+'rd'
WHEN RIGHT(Day,1) in (4,5,6,7,8,9,0) THEN Convert(varchar(2),Day)+'th'
ELSE Convert(varchar(2),Day)
END AS DaySuffix, 

WeekDayName AS DayName, 
Weekday AS DayOfWeekUSA, 
WeekDayUK AS DayOfWeekUK,
CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END AS DayOfWeekInMonth,
DayOfWeekInYear,
DayOfQuarter,
DayOfYear,
COUNT(*) OVER (PARTITION BY Year, Month,   (CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END) ORDER BY Year,Month,DayOfYear) AS WeekOfMonth,
COUNT(*) OVER (PARTITION BY Year, Quarter, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END) ORDER BY Year,Quarter,DayOfYear) AS WeekOfQuarter,
--COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END) ORDER BY DateKey) AS WeekOfYear,
COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END)   ORDER BY Year,DayOfYear) AS WeekOfYear,

WeekEndingDate_Sunday,
 Month, MonthName, 
 CASE 
 WHEN Quarter = 1 THEN MONTH
 WHEN Quarter = 2 THEN MONTH - 3
 WHEN Quarter = 3 THEN MONTH - 6
 WHEN Quarter = 4 THEN MONTH - 9
 END AS MonthOfQuarter,
 Quarter, QuarterName,Year,
 'CY '+CONVERT(VARCHAR(4),Year) AS YearName,
 FORMAT([Date], 'MMM', 'en-US')+'-'+CONVERT(VARCHAR(4),Year) AS MonthYear,
 CONVERT(VARCHAR(2),Month)+CONVERT(VARCHAR(4),Year) AS MMYYYY,
 FirstDayOfMonth, LastDayOfMonth,
 FirstDayOfQuarter, LastDayOfQuarter,
 FirstDayOfYear, LastDayOfYear,
 IsHoliday AS IsHolidayUSA, 
 IsWorkDay AS IsWeekDay, 
 HolidayText AS HolidayUSA,
 IsHolidayUK,
 HolidayTextUK AS HolidayUK,
 DayOfYear AS FiscalDayOfYear,
 COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END)   ORDER BY Year,DayOfYear) AS FiscalWeekOfYear,
 Month AS FiscalMonth,
 Quarter AS FiscalQuarter,
 QuarterName AS FiscalQuarterName,
 Year AS FiscalYear,
 'FY '+CONVERT(VARCHAR(4),Year) AS FiscalYearName,
  FORMAT([Date], 'MMM', 'en-US')+'-'+RIGHT(CONVERT(VARCHAR(4),Year),2) AS FiscalMonthYear,
  CONVERT(VARCHAR(2),Month)+CONVERT(VARCHAR(4),Year) AS FiscalMMYYYY,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfMonth),1,'USA'),101)  AS FiscalFirstDayOfMonth,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfMonth),1,'USA'),101) AS FiscalLastDayOfMonth,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfQuarter),1,'USA'),101) AS FiscalFirstDayOfQuarter,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfQuarter),1,'USA'),101)  AS FiscalLastDayOfQuarter,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfYear),1,'USA'),101) AS FiscalFirstDayOfYear,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfYear),1,'USA'),101) AS FiscalLastDayOfYear,
  '??' AS PERIOD_CODE,
  
  CASE WHEN [Date] = CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfMonth),1,'USA'),101)  THEN 1 
  ELSE 0
  END AS Day1MonthEnd,
  
  CASE 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 AND MonthWorkLeft = 1  THEN MonthWorkDay+1 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN MonthWorkDay 
  ELSE NULL 
  END AS InvoiceDay ,
  
  CASE
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN CONVERT(CHAR(8),dbo.ufnPreviousWorkdays([date],1,'USA'),112) 
  ELSE NULL 
  END AS Workday_Date_DWKey,
  
  CASE 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN dbo.ufnGetWorkDaysbyRange(CONVERT(VARCHAR(2),MONTH(dbo.ufnPreviousWorkdays([date],1,'USA'))) +'/1/'+CONVERT(VARCHAR(4),YEAR(dbo.ufnPreviousWorkdays([date],1,'USA'))),dbo.ufnPreviousWorkdays([date],1,'USA'),'USA')
  ELSE NULL
  END AS Work_Day
  
  --dbo.ufnPreviousWorkdays([date],1,'USA') AS PrevWorkDay
 -- '-----------' AS [Space],
 -- WeekOfMonth,  WeekOfYear, 
 --ISOWeekOfYear,  WeekNo, MonthNo, QuaterlyNo,  IsWeekend, FirstDayOfWeek, LastDayOfWeek,  MonthWorkDay, MonthWorkLeft, MonthWorkTotal, QuarterWorkDay, QuarterWorkLeft, QuarterWorkTotal, YearWorkDay, YearWorkLeft, YearWorkTotal, CalendarDate
FROM            dbo.dimCalendar
) AS A
ORDER BY [Date];


-- Output new Inserted Calendar Dates.
SELECT * FROM Flavors_DW.DM.DIM_DATE WHERE YEAR = @VarYear ORDER BY DateKey
RETURN
END

-- ########################################################################
-- Just Output the Generated Calendar
-- ########################################################################

IF @InsertYN = 'N'
BEGIN 
SELECT DateKey, Date, FullDateUK, FullDateUSA, DayOfMonth, DaySuffix, DayName, DayOfWeekUSA, DayOfWeekUK, DayOfWeekInMonth, DayOfWeekInYear, DayOfQuarter, DayOfYear, WeekOfMonth, WeekOfQuarter, WeekOfYear, 
                         WeekEndingDate_Sunday, Month, MonthName, MonthOfQuarter, Quarter, QuarterName, Year, YearName, MonthYear, MMYYYY, FirstDayOfMonth, LastDayOfMonth, FirstDayOfQuarter, LastDayOfQuarter, FirstDayOfYear, 
                         LastDayOfYear, IsHolidayUSA, IsWeekday, HolidayUSA, IsHolidayUK, HolidayUK, FiscalDayOfYear, FiscalWeekOfYear, FiscalMonth, FiscalQuarter, FiscalQuarterName, FiscalYear, FiscalYearName, FiscalMonthYear, 
                         FiscalMMYYYY, FiscalFirstDayOfMonth, FiscalLastDayOfMonth, FiscalFirstDayOfQuarter, FiscalLastDayOfQuarter, FiscalFirstDayOfYear, FiscalLastDayOfYear, PERIOD_CODE, Day1MonthEnd, InvoiceDay, 
                         WORKDAY_DATE_DWKEY, WORK_DAY
						 --,[Space], ISOWeekOfYear,  WeekNo, MonthNo, QuaterlyNo,  IsWeekend, FirstDayOfWeek, LastDayOfWeek,  MonthWorkDay, MonthWorkLeft, MonthWorkTotal, QuarterWorkDay, QuarterWorkLeft, QuarterWorkTotal, YearWorkDay, YearWorkLeft, YearWorkTotal, CalendarDate
FROM (
SELECT   
DateKey, CONVERT(DATETIME,DATE,101) AS [Date], 
CONVERT(VARCHAR(10), DATE,101) AS FullDateUK, 
CONVERT(VARCHAR(10), DATE,101)AS FullDateUSA,
Day AS DayOfMonth, 

CASE 
WHEN LEN(Day) = 2 AND LEFT(Day,1) = 1 THEN Convert(varchar(2),Day)+'th'
WHEN RIGHT(Day,1) = 1 THEN Convert(varchar(2),Day)+'st'
WHEN RIGHT(Day,1) = 2 THEN Convert(varchar(2),Day)+'nd'
WHEN RIGHT(Day,1) = 3 THEN Convert(varchar(2),Day)+'rd'
WHEN RIGHT(Day,1) in (4,5,6,7,8,9,0) THEN Convert(varchar(2),Day)+'th'
ELSE Convert(varchar(2),Day)
END AS DaySuffix, 

WeekDayName AS DayName, 
Weekday AS DayOfWeekUSA, 
WeekDayUK AS DayOfWeekUK,
CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END AS DayOfWeekInMonth,
DayOfWeekInYear,
DayOfQuarter,
DayOfYear,
COUNT(*) OVER (PARTITION BY Year, Month,   (CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END) ORDER BY Year,Month,DayOfYear) AS WeekOfMonth,
COUNT(*) OVER (PARTITION BY Year, Quarter, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END) ORDER BY Year,Quarter,DayOfYear) AS WeekOfQuarter,
--COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [Day] % 7 = 0 THEN 7 ELSE [Day] % 7 END) ORDER BY DateKey) AS WeekOfYear,
COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END)   ORDER BY Year,DayOfYear) AS WeekOfYear,

WeekEndingDate_Sunday,
 Month, MonthName, 
 CASE 
 WHEN Quarter = 1 THEN MONTH
 WHEN Quarter = 2 THEN MONTH - 3
 WHEN Quarter = 3 THEN MONTH - 6
 WHEN Quarter = 4 THEN MONTH - 9
 END AS MonthOfQuarter,
 Quarter, QuarterName,Year,
 'CY '+CONVERT(VARCHAR(4),Year) AS YearName,
 FORMAT([Date], 'MMM', 'en-US')+'-'+CONVERT(VARCHAR(4),Year) AS MonthYear,
 CONVERT(VARCHAR(2),Month)+CONVERT(VARCHAR(4),Year) AS MMYYYY,
 FirstDayOfMonth, LastDayOfMonth,
 FirstDayOfQuarter, LastDayOfQuarter,
 FirstDayOfYear, LastDayOfYear,
 IsHoliday AS IsHolidayUSA, 
 IsWorkDay AS IsWeekDay, 
 HolidayText AS HolidayUSA,
 IsHolidayUK,
 HolidayTextUK AS HolidayUK,
 DayOfYear AS FiscalDayOfYear,
 COUNT(*) OVER (PARTITION BY Year, (CASE WHEN [DayOfYear] % 7 = 0 THEN 7 ELSE [DayOfYear] % 7 END)   ORDER BY Year,DayOfYear) AS FiscalWeekOfYear,
 Month AS FiscalMonth,
 Quarter AS FiscalQuarter,
 QuarterName AS FiscalQuarterName,
 Year AS FiscalYear,
 'FY '+CONVERT(VARCHAR(4),Year) AS FiscalYearName,
  FORMAT([Date], 'MMM', 'en-US')+'-'+RIGHT(CONVERT(VARCHAR(4),Year),2) AS FiscalMonthYear,
  CONVERT(VARCHAR(2),Month)+CONVERT(VARCHAR(4),Year) AS FiscalMMYYYY,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfMonth),1,'USA'),101)  AS FiscalFirstDayOfMonth,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfMonth),1,'USA'),101) AS FiscalLastDayOfMonth,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfQuarter),1,'USA'),101) AS FiscalFirstDayOfQuarter,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfQuarter),1,'USA'),101)  AS FiscalLastDayOfQuarter,
  CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfYear),1,'USA'),101) AS FiscalFirstDayOfYear,
  CONVERT(DATE,dbo.ufnPreviousWorkdays(DATEADD(d,1,LastDayOfYear),1,'USA'),101) AS FiscalLastDayOfYear,
  '??' AS PERIOD_CODE,
  
  CASE WHEN [Date] = CONVERT(DATE,dbo.ufnNextWorkdays(DATEADD(d,-1,FirstDayOfMonth),1,'USA'),101)  THEN 1 
  ELSE 0
  END AS Day1MonthEnd,
  
  CASE 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 AND MonthWorkLeft = 1  THEN MonthWorkDay+1 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN MonthWorkDay 
  ELSE NULL 
  END AS InvoiceDay ,
  
  CASE
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN CONVERT(CHAR(8),dbo.ufnPreviousWorkdays([date],1,'USA'),112) 
  ELSE NULL 
  END AS Workday_Date_DWKey,
  
  CASE 
  WHEN ISWorkDay = 1 AND ISHoliday = 0 THEN dbo.ufnGetWorkDaysbyRange(CONVERT(VARCHAR(2),MONTH(dbo.ufnPreviousWorkdays([date],1,'USA'))) +'/1/'+CONVERT(VARCHAR(4),YEAR(dbo.ufnPreviousWorkdays([date],1,'USA'))),dbo.ufnPreviousWorkdays([date],1,'USA'),'USA')
  ELSE NULL
  END AS Work_Day
  
  --dbo.ufnPreviousWorkdays([date],1,'USA') AS PrevWorkDay
 -- '-----------' AS [Space],
 -- WeekOfMonth,  WeekOfYear, 
 --ISOWeekOfYear,  WeekNo, MonthNo, QuaterlyNo,  IsWeekend, FirstDayOfWeek, LastDayOfWeek,  MonthWorkDay, MonthWorkLeft, MonthWorkTotal, QuarterWorkDay, QuarterWorkLeft, QuarterWorkTotal, YearWorkDay, YearWorkLeft, YearWorkTotal, CalendarDate
FROM            dbo.dimCalendar
) AS A

ORDER BY [Date]
END 